In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import util as ut
from sklearn.neighbors import KDTree
import os 

### Data оруулах

In [12]:
df = pd.read_csv('input/output.csv')
df[df['office_name'].str.contains('Misheel city', case=False, na=False)]

,ad_id,district,khoroo,mylocation,office_name,price,area,price_m2,progress_cons,title,ad_text,clean_coords,url,date,time
9,9210383,Хан-Уул,NaN,Мишээл,Misheel city,1155.00,175,6.60,Ашиглалтад орсон,Мишээл сити м3 тауэр оффис,- Мишээл ситид оффис худалдана\n- М3 блокт биз...,47.891921 106.887409,https://www.unegui.mn/adv/9210383_misheel-siti...,6/16/2025,17:48
13,9450279,Хан-Уул,NaN,Мишээл,Misheel city,689.00,106,6.50,Ашиглалтад орсон,Мишээл сити оффис 106мк талбай,Бартер сонсоно.,47.891921 106.887409,https://www.unegui.mn/adv/9450279_misheel-siti...,6/15/2025,22:31
32,9216881,Хан-Уул,2.0,19-р хороолол,Misheel city,795.00,106,7.50,Ашиглалтад орсон,Мишээл сити м1-т оффис,Ашиглалтанд орсон засвар хийсэн 106.53м2 оффис...,47.893878 106.887541,https://www.unegui.mn/adv/9216881_misheel-siti...,6/15/2025,22:12
44,9354915,Хан-Уул,NaN,Мишээл,Misheel city,617.26,88,7.01,Ашиглалтад орсон,Мишээл бизнес тауэрт ариун цэврийн өрөөтэй офф...,Мишээл Сити төслийн М2 тауэрт өндөр давхарт до...,47.891921 106.887409,https://www.unegui.mn/adv/9354915_misheel-bizn...,5/28/2025,9:56
75,9301075,Хан-Уул,20.0,Зайсан хойно,Misheel city,352.80,56,6.30,Ашиглалтад орсон,Худ мишээл сити товерт 56 мкв оффис талбай,Хан-Уул дүүрэг -Мишээл экспо\nMisheel city M2 ...,47.8919232454507 106.887139941548,https://www.unegui.mn/adv/9301075_khan-uul-duu...,6/19/2025,21:37
83,9456798,Хан-Уул,NaN,Мишээл,Misheel city,383.29,56,6.84,Ашиглалтад орсон,"Мишээл сити цогцор, м2 тауэрт 56,21 мкв оффис","🏢ХУД, 𝐌𝐈𝐒𝐇𝐄𝐄𝐋 СITY цогцолборын М2 𝐓𝐎𝐖𝐄𝐑-т оффи...",47.891921 106.887409,https://www.unegui.mn/adv/9456798_misheel-siti...,6/19/2025,17:41
104,9286855,Хан-Уул,NaN,Мишээл,Misheel city,689.00,106,6.50,Ашиглалтад орсон,Мишээл бизнес тауэрт оффисын 106.53м.кв талбай,🌇Улаанбаатар хотын хамгийн том төсөл болох Миш...,47.891921 106.887409,https://www.unegui.mn/adv/9286855_misheel-bizn...,6/19/2025,12:59
140,8842380,Хан-Уул,NaN,Мишээл,Misheel city,803.00,105,7.65,Ашиглалтад орсон,Худ мишээл ситид оффис 105.65мк талбай,"Mишээл сити M3-т\n105,65м2 оффис, үйлчилгээний...",47.891921 106.887409,https://www.unegui.mn/adv/8842380_misheel-siti...,6/16/2025,20:46
144,9453662,Хан-Уул,NaN,Мишээл,Misheel city,380.80,56,6.80,Ашиглалтад орсон,Худ 3-р хороо мишээл сити misheel city m2 towe...,🏢ХУД 120 мянгатын Тэмээтэй хөшөөнөөс 2км зайд ...,47.891921 106.887409,https://www.unegui.mn/adv/9453662_khud-3-r-kho...,6/16/2025,20:15
145,9453580,Хан-Уул,NaN,Мишээл,Misheel city,699.60,88,7.95,Ашиглалтад орсон,Худ 3-р хороо мишээл сити misheel city m3 towe...,🏢ХУД 120 мянгатын Тэмээтэй хөшөөнөөс 2км зайд ...,47.891921 106.887409,https://www.unegui.mn/adv/9453580_khud-3-r-kho...,6/16/2025,19:53


Оффисын нэрээр дундаж үнэ болон зарын тоог харуулах

In [13]:
# mytitle дээр бүлэглээд дундаж үнэ ба зарын тоог хамт гаргах
summary = df.groupby('office_name').agg(
    avg_price_m2=('price_m2', 'mean'),
    ad_count=('price_m2', 'count')
).sort_values('avg_price_m2', ascending=False)

# Хамгийн өндөр үнэтэй 10 оффисыг харах
summary.head(10)

,avg_price_m2,ad_count
office_name,,
Minister Tower,14.292500,4
Park place,13.000000,1
Olympic residence,12.640000,1
Ayud tower,12.000000,1
Soyombo tower,11.082727,11
MERU tower,10.650000,1
Gem castle,10.560000,5
Regis place,10.284375,16
Galaxy tower,10.000000,2


### Оффис үнэлгээний аргачлал

In [21]:
df1 = pd.read_excel('D:/Projects/Office/03_Analyz/input/Office аргачлал.xlsx', sheet_name=4)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   №                190 non-null    float64
 1   district         193 non-null    object 
 2   khoroo           193 non-null    int64  
 3   number           187 non-null    object 
 4   zip code         165 non-null    float64
 5   Хаяг нэгтгэсэн   0 non-null      float64
 6   office_name      184 non-null    object 
 7   Бүтээц           193 non-null    object 
 8   floor            192 non-null    float64
 9   op_date          183 non-null    object 
 10  Гудамжны нэр     187 non-null    object 
 11  Координат        182 non-null    object 
 12  Үнэлгээ          2 non-null      float64
 13  Үнэлгээ
2024.12  2 non-null      float64
 14  Large            2 non-null      float64
 15  Байрны дугаар    0 non-null      float64
dtypes: float64(8), int64(1), object(7)
memory usage: 24.3+ KB


In [38]:
# 1. df-ээс mytitle бүрийн дундаж үнэ гаргаж dict болгон хувиргана
avg_price_dict = df.groupby('office_name')['price_m2'].mean().to_dict()

# 2. df1['office_name']-ийг ашиглан дундаж үнийг 'Үнэлгээ' баганад оруулах
df1['Үнэлгээ'] = df1['office_name'].map(avg_price_dict)
# 'result' фолдер үүсгэх
folder_path = "result"
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Файлыг 'result' фолдерт хадгалах
file_path = os.path.join(folder_path, "average_price.csv")
df1.to_csv(file_path, index=False,  encoding='utf-8-sig')

In [23]:
# mytitle дээр бүлэглээд дундаж үнэ ба зарын тоог хамт гаргах
summary = df.groupby('office_name').agg(
    avg_price_m2=('price_m2', 'min'),
    ad_count=('price_m2', 'count')
).sort_values('avg_price_m2', ascending=False)

# Хамгийн өндөр үнэтэй 10 оффисыг харах
summary.head(10)

,avg_price_m2,ad_count
office_name,,
Minister Tower,13.47,4
Park place,13.00,1
Olympic residence,12.64,1
Ayud tower,12.00,1
MERU tower,10.65,1
Galaxy tower,10.00,2
Paragon,10.00,1
Gem castle,9.80,5
Soyombo tower,9.60,11


In [24]:
df1[df1['office_name'].str.contains('Misheel city', case=False, na=False)]

,№,district,khoroo,number,zip code,Хаяг нэгтгэсэн,office_name,Бүтээц,floor,op_date,Гудамжны нэр,Координат,Үнэлгээ,Үнэлгээ\n2024.12,Large,Байрны дугаар
175,170.0,ХУД,20,75/1,17042.0,NaN,Misheel city,Төмөр бетон арагт,22.0,2024,Чингисийн өргөн чөлөө,"47.894416, 106.886189",NaN,NaN,NaN,NaN
176,171.0,ХУД,20,75/2,17042.0,NaN,Misheel city,Төмөр бетон арагт,22.0,2024,Чингисийн өргөн чөлөө,"47.893977, 106.886607",NaN,NaN,NaN,NaN
177,172.0,ХУД,20,75/3,17042.0,NaN,Misheel city,Төмөр бетон арагт,22.0,2024,Чингисийн өргөн чөлөө,"47.893380, 106.887144",NaN,NaN,NaN,NaN


In [34]:
import folium

# Газрын зургийн төв координат
map_center = [47.918873, 106.917701]
m = folium.Map(location=map_center, zoom_start=13)

# Marker нэмэх
for idx, row in df1.iterrows():
    coord = row['Координат']
    if isinstance(coord, str) and ',' in coord:
        try:
            lat, lon = map(float, coord.split(','))
            
            # Popup мэдээлэл
            popup_text = f"""
            <b>Office name:</b> {row.get('office_name', '')}<br>
            <b>Дүүрэг:</b> {row.get('district', '')}<br>
            <b>Хороо:</b> {row.get('khoroo', '')}<br>
            <b>Байрны дугаар:</b> {row.get('number', '')}<br>
            <b>Ашиглалтад орсон:</b> {row.get('op_date', '')}<br>            
            <b>Давхар:</b> {row.get('floor', '')}<br>
            <b>Үнэлгээ:</b> {row.get('Үнэлгээ', '')}
            """
            
            folium.Marker(
                location=[lat, lon],
                popup=folium.Popup(popup_text, max_width=300),
                icon=folium.Icon(color='blue', icon='building')
            ).add_to(m)
        except ValueError:
            print(f"Invalid coordinate: {coord}")

# Газрын зургийг HTML файлд хадгалах
m.save("office_map_with_details.html")

In [33]:
# 1. Координат баганаас lat, lon-г задлах
df1[['lat', 'lon']] = df1['Координат'].str.extract(r'([0-9\.]+),\s*([0-9\.]+)').astype(float)

# 2. Үнэлгээтэй оффисууд
df_valid = df1[df1['Үнэлгээ'].notnull() & df1['lat'].notnull() & df1['lon'].notnull()]
valid_coords = df_valid[['lat', 'lon']].values
valid_prices = df_valid['Үнэлгээ'].values

# 3. KDTree үүсгэх
tree = KDTree(valid_coords)

# 4. Үнэлгээ байхгүй мөрүүд
missing_mask = df1['Үнэлгээ'].isnull() & df1['lat'].notnull() & df1['lon'].notnull()
missing_df = df1[missing_mask]

# 5. Хамгийн ойр 5 оффисын дундаж үнэлгээгээр нөхөх
k = 5
for idx, row in missing_df.iterrows():
    dist, ind = tree.query([[row['lat'], row['lon']]], k=k)
    nearest_prices = valid_prices[ind[0]]
    df1.at[idx, 'Үнэлгээ'] = np.mean(nearest_prices)


In [37]:
# 'result' фолдер үүсгэх
folder_path = "result"
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Файлыг 'result' фолдерт хадгалах
file_path = os.path.join(folder_path, "filled_price.csv")
df1.to_csv(file_path, index=False,  encoding='utf-8-sig')